## Read file

In [153]:
import csv
import re

rows = []
header = []

filename = 'laptops.csv'

with open(filename, newline='') as file:
    csv_reader = csv.reader(file, delimiter=',')
    header = next(csv_reader)
    for idx, row in enumerate(csv_reader):
        if idx > 4:
            break
        rows.append(row)
        
print(header)

print(rows)

['Id', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']
[['6571244', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339'], ['7287764', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898'], ['3362737', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575'], ['9722156', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537'], ['8550527', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB 

## Inventory class

In [154]:
def row_price(row):
    return row[-1]

class Inventory:
    def __init__(self, csv_filename):
        with open(csv_filename, newline='') as file:
            csv_reader = csv.reader(file, delimiter=',')
            self.header = next(csv_reader)
            self.rows = []
            for row in csv_reader:
                self.rows.append(row)
                row[-1] = int(row[-1])
                
        self.id_to_row = {}
        for row in rows:
            self.id_to_row[row[0]] = row
            
        self.prices = set()
        for row in rows:
            self.prices.add(row[-1])
            
        self.rows_by_price = sorted(self.rows, key = row_price)
        
    def get_laptop_from_id(self, laptop_id):
        for laptop in self.rows:
            if laptop_id == laptop[0]:
                return laptop
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for laptop in self.rows:
            if laptop[-1] == dollars:
                return True
        for l1 in range(len(self.rows)):
            for l2 in range(l1 + 1, len(self.rows)):
                if self.rows[l1][-1] + self.rows[l2][-1] == dollars:
                    return True
        return False
    
    def check_promotion_dollars_fast(self, dollars):
        seen = set()
        if dollars in self.prices:
            return True
        for laptop in self.rows:
            if dollars - laptop[-1] in seen:
                return True
            seen.add(laptop[-1])   
        return False
    
    def find_first_laptop_more_expensive(self, target_price):
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                   
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price > target_price:
                range_end = range_middle
            else:
                range_start = range_middle + 1
        if self.rows_by_price[range_start][-1] <= target_price:                  
            return -1                                   
        return range_start
        
    """ 
    Query that finds all laptops whose price is in the given range.
    
    Args:
        min_price: price lower bound
        max_price: price upper bound
    """
    def get_laptops_with_price_within(self, min_price, max_price):
        return [laptop for laptop in self.rows_by_price if laptop[-1] <= max_price and laptop[-1] >= min_price]

    """
    Query that finds the cheapest laptop that matches the desired 
    RAM and memory capacity

    Args:
        ram: string that represents RAM's capacity
        memory: string that represents the hard drive capacity
    """
    def _get_spec_from_ram(self, spec):
        return int(re.sub(r'[a-zA-Z]','',spec))
        
    def _get_spec_from_memory(self, string):
        mask = r'(\d+[A-Za-z]+)\s*([A-Za-z\s]+)'
        matches = re.findall(mask, string)
        tuples = set()
        for match in matches:
            amount_storage = match[0]
            type_storage = match[1].strip()
            tuples.add((amount_storage, type_storage))
        return tuples
        
    def find_the_cheapest_laptop_with(self, ram, memory):
        idx = 0
        idx_cheapest = None
        price_cheapest = self.rows_by_price[0][-1]
        
        desired_ram = self._get_spec_from_ram(ram)
        desired_memory = self._get_spec_from_memory(memory)
        
        for laptop in self.rows_by_price:
            target_ram = self._get_spec_from_ram(laptop[7])
            target_memory = self._get_spec_from_memory(laptop[8])
            if desired_ram == target_ram and desired_memory == target_memory:
                if laptop[-1] < price_cheapest or idx_cheapest is None:
                    price_cheapest = laptop[-1]
                    idx_cheapest = idx
            idx += 1 
            
        if idx_cheapest is None:
            return -1
            
        return self.rows_by_price[idx_cheapest]

## Testing

First, we instantiate the class

In [155]:
inventory = Inventory(filename)

Then, we perform the queries

In [156]:
print("\n", "Laptops whose prices are within 100 and 200")
print(inventory.get_laptops_with_price_within(100,200))

print("\n", "Cheapest laptop with 8GB RAM and 256GB SSD")
print(inventory.find_the_cheapest_laptop_with('8GB','256GB SSD'))

print("\n", "Cheapest laptop with 2GB RAM and 16GB SSD") 
print(inventory.find_the_cheapest_laptop_with('2GB','16GB SSD'))


 Laptops whose prices are within 100 and 200
[['3564228', 'Acer', 'C740-C9QX (3205U/2GB/32GB/Chrome', 'Netbook', '11.6', '1366x768', 'Intel Celeron Dual Core 3205U 1.5GHz', '2GB', '32GB SSD', 'Intel HD Graphics', 'Chrome OS', '1.3kg', 174], ['7667029', 'Asus', 'Vivobook E200HA', 'Netbook', '11.6', '1366x768', 'Intel Atom x5-Z8350 1.44GHz', '2GB', '32GB Flash Storage', 'Intel HD Graphics 400', 'Windows 10', '0.98kg', 191], ['1478754', 'Vero', 'V131 (X5-Z8350/4GB/32GB/FHD/W10)', 'Notebook', '13.3', 'Full HD 1920x1080', 'Intel Atom X5-Z8350 1.44GHz', '4GB', '32GB Flash Storage', 'Intel HD Graphics 400', 'Windows 10', '1.35kg', 196], ['4366200', 'Asus', 'E402WA-GA010T (E2-6110/2GB/32GB/W10)', 'Notebook', '14', '1366x768', 'AMD E-Series E2-6110 1.5GHz', '2GB', '32GB Flash Storage', 'AMD Radeon R2', 'Windows 10', '1.65kg', 199], ['3840240', 'Acer', 'Chromebook C910-C2ST', 'Notebook', '15.6', '1366x768', 'Intel Celeron Dual Core 3205U 1.5GHz', '2GB', '16GB SSD', 'Intel HD Graphics', 'Chrome 